In [1]:
%matplotlib inline

import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt

from collections import defaultdict
from scipy.stats.stats import pearsonr

import fim
from fim import apriori

In [2]:
# NOTE: Function whose goal is to transform the dataset 'transactions.csv' from the relational format
#       to the transactional format. This can be ignored for two reasons: 
#          a) the dataset 'transactions.csv' was never published on the website
#          b) we're going to transform the HR dataset in transactional format later on, without the help of this

# Converts data from relational style to transactional style
def rel2tra(fileinput, fileoutput, delimiter=',', has_header=True):
    data = open(fileinput, 'r')
    if has_header:
        data.readline()
    baskets = defaultdict(list)

    for row in data:
        basket_id = row.replace('\r\n', '').split(delimiter)[0]
        item_id = row.replace('\r\n', '').split(delimiter)[1]
        baskets[basket_id].append(item_id)

    data.close()

    out = open(fileoutput, 'w')
    for k, v in baskets.iteritems():
        s = '%s' % k
        for item in v:
            s += ',%s' % item
        out.write('%s\n' % s)
    out.close()
    
    return baskets

In [3]:
# NOTE: please ignore, see above
baskets = rel2tra('transactions.csv', 'baskets.csv', delimiter=',', has_header=True)

IOError: [Errno 2] No such file or directory: 'transactions.csv'

In [127]:
# NOTE: please ignore, see above
baskets.keys()[:10]

NameError: name 'baskets' is not defined

In [5]:
# NOTE: please ignore, see above
baskets['2663064031721']

['1108', '1140', '2017', '4180', '5030', '5064']

In [12]:
# NOTE: please ignore, see above
baskets_lists = [b for b in baskets.values()]

In [13]:
# NOTE: please ignore, see above
baskets_lists[:3]

[['1108', '1140', '2017', '4180', '5030', '5064'],
 ['1122', '2243', '2551', '3569', '3664', '3000442'],
 ['437', '559']]

In [74]:
# Sample call to 'apriori'. This is not going to work anyway, see notes above
itemsets = apriori(baskets_lists[:100], supp=2, zmin=2, target='a') 

In [75]:
# NOTE: please ignore, see above
itemsets

[(('1658', '2650'), 2),
 (('3000442', '2551'), 2),
 (('2061', '3672'), 2),
 (('3086', '3087'), 2),
 (('5086', '2729', '445'), 2),
 (('5086', '2729'), 2),
 (('5086', '445'), 2),
 (('2674', '2650'), 2),
 (('608', '3087'), 2),
 (('2665', '2052'), 2),
 (('2665', '3828'), 2),
 (('2665', '3087'), 2),
 (('5072', '920'), 2),
 (('5025', '445'), 2),
 (('2058', '3448'), 2),
 (('2058', '445'), 2),
 (('3749', '3448'), 2),
 (('2080', '437'), 2),
 (('632', '2650'), 2),
 (('147', '1640'), 2),
 (('147', '2243'), 2),
 (('5069', '3750'), 2),
 (('441', '2193', '2504'), 2),
 (('441', '2193'), 2),
 (('441', '2504'), 2),
 (('2532', '4658'), 2),
 (('954', '3448'), 2),
 (('954', '4049'), 2),
 (('4029', '207'), 2),
 (('577', '2198'), 2),
 (('1257', '2009'), 2),
 (('4180', '2050'), 2),
 (('2494', '3448'), 2),
 (('1281', '2089'), 2),
 (('3690', '2727', '2729'), 2),
 (('3690', '2727'), 2),
 (('3690', '2729'), 2),
 (('4047', '2727', '2729'), 2),
 (('4047', '2727'), 2),
 (('4047', '2729'), 2),
 (('2495', '4776'), 2)

In [22]:
# Another sample call to 'apriori'. Again, it won't compile
rules = apriori(baskets_lists[:100], supp=3, zmin=2, target='r', 
                conf=90, report='ascl') 

In [23]:
rules

[('4049', ('3449', '2504'), 3, 0.03, 1.0, 20.0),
 ('2504', ('3449', '4049'), 3, 0.03, 1.0, 25.0),
 ('3449', ('2504', '4049'), 3, 0.03, 1.0, 33.333333333333336),
 ('2504', ('3449',), 3, 0.03, 1.0, 25.0),
 ('4049', ('3449',), 3, 0.03, 1.0, 20.0),
 ('2089', ('2443', '445'), 3, 0.03, 1.0, 8.333333333333334)]

In [18]:
help(fim.apriori)

Help on built-in function apriori in module fim:

apriori(...)
    apriori (tracts, target='s', supp=10, zmin=1, zmax=None, report='a',
             eval='x', agg='x', thresh=10, prune=None, algo='b', mode='',
             border=None)
    Find frequent item sets with the Apriori algorithm.
    tracts  transaction database to mine (mandatory)
            The database must be an iterable of transactions;
            each transaction must be an iterable of items;
            each item must be a hashable object.
            If the database is a dictionary, the transactions are
            the keys, the values their (integer) multiplicities.
    target  type of frequent item sets to find     (default: s)
            s/a   sets/all   all     frequent item sets
            c     closed     closed  frequent item sets
            m     maximal    maximal frequent item sets
            g     gens       generators
            r     rules      association rules
    supp    minimum support of an i

In [ ]:
# Calling external C function

In [5]:
import subprocess

In [6]:
# Helper function to call 'apriori' from within a linux executable file
def call_apriori(fileinput, fileoutput, delimiter=',', target_type='s', 
                 min_nbr_items=1, min_sup=2, min_conf=2):
    # apriori
    # -t# {m: maximal, c: closed, s: frequent, r: association rules}
    # -m# minimum number of items per item set/association rule
    # -s# minimum support of an item set, positive: percentage, negative: absolute
    # -c# minimum confidence rule percentage
    # -b# line delimiter (,)
    # The default additional information output format for rules is " (%X, %C)"
    # %X relative body set support as a percentage
    # %C rule confidence as a percentage
    # %L lift

    if target_type == 'r':
        call_cmd = ['./apriori', '-b%s' % delimiter, '-t%s' % target_type, '-m%s' % min_nbr_items, 
                    '-s%s' % min_sup, '-c%s' % min_conf, '-v (%X, %C, %L)', 
                    fileinput, fileoutput]
    else:
        call_cmd = ['./apriori', '-b%s' % delimiter, '-t%s' % target_type, 
                           '-m%s' % min_nbr_items, '-s%s' % min_sup, fileinput, fileoutput]

    ret = subprocess.call(call_cmd,  stdout=open('apriori_stdout.txt', 'w'), 
                          stderr=open('apriori_stderr.txt', 'w'))
    return ret

In [26]:
# NOTE: this is not going to compile

delimiter=','
target_type='s'
min_nbr_items=3
min_sup=1

ret_val = call_apriori('baskets.csv', 'freq_patterns.txt', 
                       delimiter, target_type, 
                       min_nbr_items, min_sup)

In [27]:
# NOTE: this is not going to compile

delimiter=','
target_type='r'
min_nbr_items=3
min_sup=2
min_conf=25

ret_val = call_apriori('baskets.csv', 'rules.txt', delimiter, target_type, 
                       min_nbr_items, min_sup, min_conf)

In [7]:
# Helper function to better visualize association rules returned by the call to 'apriori' (with target_type='r')

def read_rules(filename):
    data = open(filename, 'r')
    rules = list()
    for row in data:
        fileds = row.rstrip('\n\r').split(' <- ')
        cons = fileds[0]
        other = fileds[1].split(' (')
        ant = other[0].split(' ')
        other2 = other[1].split(', ')
        sup = float(other2[0])
        conf = float(other2[1])
        lift = float(other2[2].replace(')', ''))
        rule = {
            'ant': ant,
            'cons': cons,
            'sup': sup,
            'conf': conf,
            'lift': lift
        }
        rules.append(rule)
    data.close()
    return rules

In [30]:
rules = read_rules('rules.txt')
for r in rules[:3]:
    print r['ant'], '-->', r['cons'], ' lift', r['lift'], ' conf', r['conf']

['2729', '2650'] --> 2727  lift 495.026  conf 26.9805
['3448', '2443'] --> 2193  lift 312.362  conf 26.0954
['2193', '2443'] --> 3448  lift 343.622  conf 25.9445


In [31]:
rules[0]

{'ant': ['2729', '2650'],
 'conf': 26.9805,
 'cons': '2727',
 'lift': 495.026,
 'sup': 2.13263}

In [84]:
###################################################################################################################
########################################### MEANINGFUL CODE STARTS HERE ###########################################
###################################################################################################################

# Frequent Pattern Mining on Human Resources Dataset

In [8]:
df = pd.read_csv("../data/HR_comma_sep.csv")

In [9]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [10]:
def clean_data(df):
    
    # Get the unique values of 'sales'
    sales = sorted(df['sales'].unique())
    
    # Generate a mapping of 'sales' from a string to a number representation    
    sales_mapping = dict(zip(sales, range(0, len(sales) + 1)))
    
    # Transform 'sales' from a string to a number representation
    df['sales_Val'] = df['sales'].map(sales_mapping).astype(int)
    
    
    # Get the unique values of 'salary'
    salaries = sorted(df['salary'].unique())
    
    # Generate a mapping of 'salary' from a string to a number representation    
    salaries_mapping = dict(zip(salaries, range(0, len(salaries) + 1)))
    
    # Transform 'salary' from a string to a number representation
    df['salary_Val'] = df['salary'].map(salaries_mapping).astype(int)
    
    
    # Drop the columns we don't need anymore
    df = df.drop(['sales', 'salary'], axis=1)
    
    return df

In [11]:
df2 = clean_data(df)

In [12]:
df2.head(10)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_Val,salary_Val
0,0.38,0.53,2,157,3,0,1,0,7,1
1,0.80,0.86,5,262,6,0,1,0,7,2
2,0.11,0.88,7,272,4,0,1,0,7,2
3,0.72,0.87,5,223,5,0,1,0,7,1
4,0.37,0.52,2,159,3,0,1,0,7,1
5,0.41,0.50,2,153,3,0,1,0,7,1
6,0.10,0.77,6,247,4,0,1,0,7,1
7,0.92,0.85,5,259,5,0,1,0,7,1
8,0.89,1.00,5,224,5,0,1,0,7,1
9,0.42,0.53,2,142,3,0,1,0,7,1


In [13]:
# Discretize 'satisfaction_level'
# (Moltiplico per 100 perché range(0, 1, 0.1) dà errore, visto che range() si aspetta parametri interi)
# NOTE: WE MAY DISCUSS ON THE MOST APPROPRIATE NUMBER OF BINS!
df2['satisfaction_level_group'] = pd.cut(df2['satisfaction_level'] * 100, bins=range(0, 120, 10), 
                                         right=False, labels=range(0, 110, 10))

In [14]:
df2.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_Val,salary_Val,satisfaction_level_group
0,0.38,0.53,2,157,3,0,1,0,7,1,30
1,0.80,0.86,5,262,6,0,1,0,7,2,80
2,0.11,0.88,7,272,4,0,1,0,7,2,10
3,0.72,0.87,5,223,5,0,1,0,7,1,70
4,0.37,0.52,2,159,3,0,1,0,7,1,30


In [15]:
# Discretize 'last_evaluation'
# NOTE: WE MAY DISCUSS ON THE MOST APPROPRIATE NUMBER OF BINS!
df2['last_evaluation_group'] = pd.cut(df2['last_evaluation'] * 100, bins=range(0, 120, 10), 
                          right=False, labels=range(0, 110, 10))

In [16]:
df2.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_Val,salary_Val,satisfaction_level_group,last_evaluation_group
0,0.38,0.53,2,157,3,0,1,0,7,1,30,50
1,0.80,0.86,5,262,6,0,1,0,7,2,80,80
2,0.11,0.88,7,272,4,0,1,0,7,2,10,80
3,0.72,0.87,5,223,5,0,1,0,7,1,70,80
4,0.37,0.52,2,159,3,0,1,0,7,1,30,50


In [17]:
# Discretize 'average_montly_hours'
# NOTE: WE MAY DISCUSS ON THE MOST APPROPRIATE NUMBER OF BINS!
df2['average_montly_hours_group'] = pd.cut(df2['average_montly_hours'], bins=range(90, 340, 20), 
                          right=False, labels=range(90, 320, 20))

In [18]:
df2.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_Val,salary_Val,satisfaction_level_group,last_evaluation_group,average_montly_hours_group
0,0.38,0.53,2,157,3,0,1,0,7,1,30,50,150
1,0.80,0.86,5,262,6,0,1,0,7,2,80,80,250
2,0.11,0.88,7,272,4,0,1,0,7,2,10,80,270
3,0.72,0.87,5,223,5,0,1,0,7,1,70,80,210
4,0.37,0.52,2,159,3,0,1,0,7,1,30,50,150


In [19]:
df2.drop(['satisfaction_level', 'last_evaluation', 'average_montly_hours'], axis=1, inplace=True)
df2.head()

,number_project,time_spend_company,Work_accident,left,promotion_last_5years,sales_Val,salary_Val,satisfaction_level_group,last_evaluation_group,average_montly_hours_group
0,2,3,0,1,0,7,1,30,50,150
1,5,6,0,1,0,7,2,80,80,250
2,7,4,0,1,0,7,2,10,80,270
3,5,5,0,1,0,7,1,70,80,210
4,2,3,0,1,0,7,1,30,50,150


In [20]:
df3 = df2
df3['number_project'] = df2['number_project'].astype(str) + '_NP'
df3['time_spend_company'] = df2['time_spend_company'].astype(str) + '_TSC'
df3['Work_accident'] = df2['Work_accident'].astype(str) + '_WA'
df3['left'] = df2['left'].astype(str) + '_L'
df3['promotion_last_5years'] = df2['promotion_last_5years'].astype(str) + '_PL5'


sales_inverse_mapping = \
{
 0 : 'IT',
 1 : 'RandD',
 2 : 'accounting',
 3 : 'hr',
 4 : 'management',
 5 : 'marketing',
 6 : 'product_mng',
 7 : 'sales',
 8 : 'support',
 9 : 'technical'
}
df3['sales_Val'] = df2['sales_Val'].map(sales_inverse_mapping).astype(str)

salary_inverse_mapping = {0: 'high', 1: 'low', 2: 'medium'}
df3['salary_Val'] = df2['salary_Val'].map(salary_inverse_mapping).astype(str)

df3['satisfaction_level_group'] = df2['satisfaction_level_group'].astype(str) + '_SL'
df3['last_evaluation_group'] = df2['last_evaluation_group'].astype(str) + '_LE'
df3['average_montly_hours_group'] = df2['average_montly_hours_group'].astype(str) + '_AMH'

In [21]:
df3.head()

,number_project,time_spend_company,Work_accident,left,promotion_last_5years,sales_Val,salary_Val,satisfaction_level_group,last_evaluation_group,average_montly_hours_group
0,2_NP,3_TSC,0_WA,1_L,0_PL5,sales,low,30_SL,50_LE,150_AMH
1,5_NP,6_TSC,0_WA,1_L,0_PL5,sales,medium,80_SL,80_LE,250_AMH
2,7_NP,4_TSC,0_WA,1_L,0_PL5,sales,medium,10_SL,80_LE,270_AMH
3,5_NP,5_TSC,0_WA,1_L,0_PL5,sales,low,70_SL,80_LE,210_AMH
4,2_NP,3_TSC,0_WA,1_L,0_PL5,sales,low,30_SL,50_LE,150_AMH


In [22]:
df3.to_csv('../data/hr_for_pattern_mining.csv', sep=',', header=False)

In [32]:
delimiter=','
target_type='s'
min_nbr_items=2
min_sup=2
min_conf=2

ret_val = call_apriori('../data/hr_for_pattern_mining.csv', 'hr_freq_patterns.txt', 
                       delimiter, target_type, min_nbr_items, min_sup, min_conf)

In [33]:
delimiter=','
target_type='r'
min_nbr_items=2
min_sup=2
min_conf=25

ret_val = call_apriori('../data/hr_for_pattern_mining.csv', 'hr_rules.txt', delimiter, target_type, 
                       min_nbr_items, min_sup, min_conf)

In [34]:
rules = read_rules('hr_rules.txt')
for r in rules[:3]:
    print r['ant'], '-->', r['cons'], ' lift', r['lift'], ' conf', r['conf']

['10_SL', '4_TSC', '1_L', 'low', '0_WA', '0_PL5'] --> 7_NP  lift 1574.3  conf 26.8698
['10_SL', '4_TSC', '1_L', 'low', '0_WA'] --> 7_NP  lift 1561.32  conf 26.6484
['10_SL', '4_TSC', '1_L', 'low', '0_PL5'] --> 7_NP  lift 1487.51  conf 25.3886


In [35]:
# Collect those rules whose consequences tell something about the 'left' attribute
rulse_cons_L = list()
for r in rules:
    if r['cons'].endswith('_L'):
        rulse_cons_L.append(r)

In [36]:
print len(rulse_cons_L)

2890


In [37]:
sorted_rules_cons_L = sorted(rulse_cons_L, key=lambda r: r['conf'], reverse=True)

In [39]:
for r in sorted_rules_cons_L[:10]:
    print r['ant'], '-->', r['cons'], ' lift', r['lift'], ' conf', r['conf']

['high', '2_TSC'] --> 0_L  lift 131.248  conf 100.0
['90_SL', '60_LE', '0_WA'] --> 0_L  lift 131.248  conf 100.0
['90_SL', '60_LE', '0_PL5'] --> 0_L  lift 131.248  conf 100.0
['90_SL', '60_LE'] --> 0_L  lift 131.248  conf 100.0
['90_SL', '50_LE', '0_PL5'] --> 0_L  lift 131.248  conf 100.0
['90_SL', '50_LE'] --> 0_L  lift 131.248  conf 100.0
['90_SL', '2_TSC', '0_WA', '0_PL5'] --> 0_L  lift 131.248  conf 100.0
['90_SL', '2_TSC', '0_WA'] --> 0_L  lift 131.248  conf 100.0
['90_SL', '2_TSC', '0_PL5'] --> 0_L  lift 131.248  conf 100.0
['90_SL', '2_TSC'] --> 0_L  lift 131.248  conf 100.0


In [40]:
df3.values[0]

array(['2_NP', '3_TSC', '0_WA', '1_L', '0_PL5', 'sales', 'low', '30_SL',
       '50_LE', '150_AMH'], dtype=object)

In [41]:
employee_test = df3.values[0]

In [43]:
for r in rules:
    if set(r['ant']) < set(employee_test) and r['cons'].endswith('_L'):
        print r['ant'], '-->', r['cons']

['30_SL', '2_NP', '50_LE', '3_TSC', '0_WA', '0_PL5'] --> 1_L
['30_SL', '2_NP', '50_LE', '3_TSC', '0_WA'] --> 1_L
['30_SL', '2_NP', '50_LE', '3_TSC', '0_PL5'] --> 1_L
['30_SL', '2_NP', '50_LE', '3_TSC'] --> 1_L
['30_SL', '2_NP', '50_LE', '0_WA', '0_PL5'] --> 1_L
['30_SL', '2_NP', '50_LE', '0_WA'] --> 1_L
['30_SL', '2_NP', '50_LE', '0_PL5'] --> 1_L
['30_SL', '2_NP', '50_LE'] --> 1_L
['30_SL', '2_NP', '3_TSC', 'low', '0_WA', '0_PL5'] --> 1_L
['30_SL', '2_NP', '3_TSC', 'low', '0_WA'] --> 1_L
['30_SL', '2_NP', '3_TSC', 'low', '0_PL5'] --> 1_L
['30_SL', '2_NP', '3_TSC', 'low'] --> 1_L
['30_SL', '2_NP', '3_TSC', '0_WA', '0_PL5'] --> 1_L
['30_SL', '2_NP', '3_TSC', '0_WA'] --> 1_L
['30_SL', '2_NP', '3_TSC', '0_PL5'] --> 1_L
['30_SL', '2_NP', '3_TSC'] --> 1_L
['30_SL', '2_NP', 'low', '0_WA', '0_PL5'] --> 1_L
['30_SL', '2_NP', 'low', '0_WA'] --> 1_L
['30_SL', '2_NP', 'low', '0_PL5'] --> 1_L
['30_SL', '2_NP', 'low'] --> 1_L
['30_SL', '2_NP', '0_WA', '0_PL5'] --> 1_L
['30_SL', '2_NP', '0_WA'] --> 1

In [44]:
hr_baskets_list = list()
for row in df3.values:
    hr_baskets_list.append(list(row))

In [45]:
rules = apriori(hr_baskets_list, supp=5, zmin=2, target='r', conf=90, report='ascl') 

In [46]:
for rule in rules:
    if rule[0] == '1_L': # i.e. If left=1 is the consequence of this association rule
        print rule

('1_L', ('40_SL', '2_NP', '3_TSC', '0_WA', '0_PL5'), 882, 0.058803920261350756, 0.9576547231270358, 4.0223643775363795)
('1_L', ('40_SL', '2_NP', '3_TSC', '0_WA'), 891, 0.0594039602640176, 0.9580645161290322, 4.024085599949413)
('1_L', ('40_SL', '2_NP', '3_TSC', '0_PL5'), 933, 0.06220414694312954, 0.9501018329938901, 3.990640546926731)
('1_L', ('40_SL', '2_NP', '3_TSC'), 945, 0.06300420028001867, 0.9507042253521126, 3.9931707297833485)
('1_L', ('40_SL', '2_NP', '0_WA', '0_PL5'), 888, 0.05920394692979532, 0.9070480081716037, 3.8098048374589424)
('1_L', ('40_SL', '2_NP', '0_WA'), 897, 0.059803986932462166, 0.9078947368421053, 3.813361287565034)
('1_L', ('130_AMH', '2_NP', '3_TSC', '0_WA', '0_PL5'), 815, 0.05433695579705314, 0.9554513481828839, 4.01310970915572)
('1_L', ('130_AMH', '2_NP', '3_TSC', '0_WA'), 817, 0.05447029801986799, 0.9555555555555556, 4.013547403466194)
('1_L', ('130_AMH', '2_NP', '3_TSC', '0_PL5'), 852, 0.05680378691912794, 0.9508928571428571, 3.9939630255630676)
('1_L'